In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [8]:
### Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
## Preprocess the data
### drop irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

## Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

## Onehot ecode 'Geography'
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

## Combine one hot encoded 'Geography' with the original dataframe
data = pd.concat([data, geo_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)

## Divide the data into features (X) and target (y)

X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
## Save the encoder and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [11]:
## Define a function to create a model and try different hyperparameters

def create_model(neurous=32, layers=1):
    model = Sequential()
    model.add(Dense(neurous, input_shape=(X_train.shape[1],), activation='relu'))
    
    for _ in range(layers):
        model.add(Dense(neurous, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
## Create a KerasClassifier wrapper for GridSearchCV

model = KerasClassifier(neurous=32, layers=1, build_fn=create_model, epochs=50, batch_size=10, verbose=1)


In [18]:
param_grid = {'neurous': [16, 32, 64], 'layers': [1, 2], 'batch_size': [10, 20]}

In [19]:
## Perform GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/50


c:\Users\praka\VSCodeProjects\python\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\praka\VSCodeProjects\python\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 897us/step - accuracy: 0.7582 - loss: 0.5283
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.8273 - loss: 0.4028
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 924us/step - accuracy: 0.8452 - loss: 0.3785
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 847us/step - accuracy: 0.8539 - loss: 0.3614
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - accuracy: 0.8497 - loss: 0.3613
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1000us/step - accuracy: 0.8558 - loss: 0.3471
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step - accuracy: 0.8637 - loss: 0.3296
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step - accuracy: 0.8551 - loss: 0.3421
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step - accuracy: 0.8629 - loss: 0.3355
Epoch 10/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 942us/step - accuracy: 0.8642 - loss: 0.3265
Epoch 11/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 826us/step - accuracy: 0.8651 - loss: 0.3271
Epoch 12/50
800/800 ━━━━━━━━━━━━━━━━━━━━